In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
import string

url = "https://raw.githubusercontent.com/sahalmaghfud/MLLawanJudi/main/datasetA.csv"
df = pd.read_csv(url)

texts = df['text'].values
labels = df['label'].values



In [3]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=10,shuffle=True)

print(f" training: {len(X_train)}")
print(f"testing: {len(X_test)}")

print(X_test)



 training: 110
testing: 10
['Personal Bisnis Perusahaan EN ID Download DANA tagline Mulai transaksi pakai #BukanDompetBiasa sekarang! Indonesia telah terdaftar dan diawasi oleh bank-indonesiakominfo dana-app showcaseshowcase DOMPET DIGITAL Seperti dompetmu, tapi lebih canggih Wallet bikin kamu mudah untuk akses semua kebutuhan berinvestasi. Cari Tahu Selengkapnya LAYANAN KEUANGAN Dijamin aman, makin dekat meraih impian Investasi eMAS terjamin aman bisa ditukar fisik, didukung lembaga penyedia yang BAPPEBTI. Pantau harga terupdatenya di DANA! Bayar CICIL Lindungi berharga dengan Siaga bebas cemas GAYA HIDUP Belanja jadi hemat voucher Temukan beli sesuai kebutuhanmu. Dapatkan informasi lengkap tentang nilai voucher, masa berlaku cara pakainya. My Bills mengatur tagihanmu menarik hanya Deals Bagikan aktivitasmu Feed showcase KEAMANAN Perlindungan Protection standar PCI DSS & ISO 27001, mengamankan data pembayaran milikmu. Garansi saat ada kehilangan uang Saldo DANA, kartu bank sumber lain

In [4]:
# Constants
MAX_LENGTH = 700
MAX_VOCAB_SIZE = 10000
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding=padding_type, truncating=trunc_type)

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding=padding_type, truncating=trunc_type)

vocab_size = len(tokenizer.word_index) + 1

print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 9110


In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(MAX_LENGTH,)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=100),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 700, 100)       │       911,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 696, 64)        │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 944,121 (3.60 MB)

 Trainable params: 944,121 (3.60 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
print("X_train_pad shape:", X_train_pad.shape)
print("y_train shape:", y_train.shape)
print("X_test_pad shape:", X_test_pad.shape)
print("y_test shape:", y_test.shape)

X_train_pad shape: (110, 700)
y_train shape: (110,)
X_test_pad shape: (10, 700)
y_test shape: (10,)


In [7]:
history = model.fit(X_train_pad, y_train, epochs=20, validation_data=(X_test_pad, y_test))

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.5895 - loss: 0.6836 - val_accuracy: 0.6000 - val_loss: 0.6823
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.5733 - loss: 0.6737 - val_accuracy: 0.9000 - val_loss: 0.6734
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.7071 - loss: 0.6442 - val_accuracy: 0.8000 - val_loss: 0.6659
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 0.7841 - loss: 0.6224 - val_accuracy: 0.9000 - val_loss: 0.6568
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.8163 - loss: 0.5814 - val_accuracy: 0.9000 - val_loss: 0.6444
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 0.9339 - loss: 0.5438 - val_accuracy: 1.0000 - val_loss: 0.6300
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step - accuracy: 0.8892 - loss: 0.5340 - val_accuracy: 1.0000 - val_loss: 0.6102
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - accuracy: 0.9433 - loss: 0.4718 - val_accuracy: 1.0000 - val_loss:

In [8]:


new_text = ["baim234: HOME Selamat Datang Kakak di Baim234 - Situs Resmi Slot Gacor Anti Rungkad Terbaik Dan Bandar Togel Terpercaya di Indonesia. Member wajib deposit ke rekening yang tertera sekarang di tujuan deposit, jika member deposit ke rekening tujuan deposit yang sebelumnya, maka tidak bisa di proses, sewaktu - waktu tujuan deposit bisa berubah tanpa pemberitahuan. Rupiah Masuk Daftar Lupa Kata Sandi ? Slot Pragmatic Play PG Soft Afb Gaming NEXTSPIN Spade Gaming FASTSPIN Fa Chai OG Slot Advantplay GB Game Playstar JDB NoLimit City HACKSAW Habanero Joker Gaming CQ9 Gaming JILI Microgaming RED TIGER Playtech Slot Casino Pragmatic Play LC OG Casino AFB CASINO WM Casino GD88 SA GAMING Evolution Dream Gaming Asia Gaming Sexy Gaming LG88 WE CASINO PLAYTECH CASINO Live Game BAIM234 LIVE COD GAMING SV388 Cockfight MIKI Gaming WS168 GA28 Sportsbook AFB88 CMD368 SBO SPORT SABA Sports OPUS Sports Lottery 4D Poker We1Poker Poker Q Arcade Spaceman Joker Gaming JILI Fastspin Fa Chai SPRIBE JDB Spade Gaming CQ9 Berita Promosi Daftar Selamat Datang Kakak di Baim234 - Situs Resmi Slot Gacor Anti Rungkad Terbaik Dan Bandar Togel Terpercaya di Indonesia. Member wajib deposit ke rekening yang tertera sekarang di tujuan deposit, jika member deposit ke rekening tujuan deposit yang sebelumnya, maka tidak bisa di proses, sewaktu - waktu tujuan deposit bisa berubah tanpa pemberitahuan. Indonesia Selamat Datang Silakan Login atau Mendaftar Masuk Daftar Lupa Password? Home Event Promosi Berita Informasi Keluaran Bantuan Kontak Kami Region and Language Indonesia Indonesia id English en Rp Game Terpopuler Tampilkan Lainnya SUGAR BANG BANG Legacy Of Kong Maxways MPOPLAY Money Tree Mahjong Wins 3 - Black Scatter Rhapsody of Muertos Mahjong Ways Mpoplay Lucky Twins Wilds FOR THE HORDE Royale House Mahjong Dragon GOLDEN GENIE Gates of Olympus Mahjong Ways 2 Pong Pong Mahjong Jackpots Mahjong Princess MPOPLAY King Arthur Aztec Gold Treasure MAHJONG WAYS 3+ Slot Tampilkan Lainnya Pragmatic Play PG Soft A"]
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=MAX_LENGTH)

prediction = model.predict(new_text_pad)
print(prediction)
print(f"Prediksi: {'Judi Online' if prediction > 0.5 else 'Bukan Judi Online'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
[[0.9815395]]
Prediksi: Judi Online


In [9]:
new_text = ["Psikolog Jelaskan Dampak Kecanduan Judi Online buat Anak Remaja Psikolog Jelaskan Dampak Kecanduan Judi Online buat Anak Remaja 0 komentar BAGIKAN Tautan telah disalin MENU detikcom Terpopuler Live TV Kirim Tulisan detikPilkada NEW Kategori Berita detikNews detikFinance detikInet detikHot detikSport Sepakbola detikOto detikTravel detikFood detikHealth Wolipop DetikX 20Detik detikFoto detikEdu detikHikmah detikProperti detikPop NEW Daerah detikJateng detikJatim detikJabar detikSulsel detikSumut detikBali detikSumbagsel detikJogja NEW Layanan Pasang Mata adsmart detikEvent Trans Snow World Trans Studio Flying Over Indonesia berbuatbaik.id ziswafctarsa.id Signature Awards NEW Bingkai.id NEW Detik Network CNN Indonesia CNBC Indonesia Hai Bunda InsertLive Beautynesia Female Daily CXO Media Cancel Yang sedang ramai dicari Loading... Terakhir yang dicari Loading... Home Sekolah Perguruan Tinggi Beasiswa Edutainment Seleksi Masuk PT Detikpedia Foto Video Infografis Indeks Most Popular Bank Soal Point of View detikEdu Sekolah Psikolog Jelaskan Dampak Kecanduan Judi Online buat Anak Remaja Trisna Wulandari - detikEdu Senin, 02 Des 2024 20:00 WIB Muzakki Ramdhan, Egy Octa Suroso, dan Baim Alkatiri di film pendek tentang judi online, Kemenangan Sejati. Foto: Instagram @cerdasberkarakter.kemdikbudri Daftar Isi Dampak Judi Online Ganggu Kesehatan Mental Stres soal Uang Hubungan Merenggang Rentan Kriminalitas Jakarta - Laporan Pusat Pelaporan dan Analisis Transaksi Keuangan (PPATK) menunjukkan tak kurang dari 197.000 anak Indonesia terlibat judi online sepanjang 2024. Anak-anak yang terpapar judi online berada di rentang usia 11-19 tahun. Psikolog Irma Gustiana mengamini bahwa remaja punya rasa ingin tahu yang besar. Karena itu, godaan untuk ingin mencoba judi online terkadang bisa muncul. Namun, ia menegaskan dampak judi online bisa berbuntut panjang. Baca juga: 960 Ribu Pelajar Terlibat Judol, Kemendiktisaintek Bakal Bantu Rehabilitasi Korban Dampak Judi Online Ganggu Kesehatan"]
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=MAX_LENGTH)

prediction = model.predict(new_text_pad)
print(prediction)
print(f"Prediksi: {'Judi Online' if prediction > 0.5 else 'Bukan Judi Online'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[0.30238333]]
Prediksi: Bukan Judi Online


In [10]:
from google.colab import files
tf.keras.models.save_model(model, 'modelLajuA.keras')
files.download('modelLajuA.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>